In [1]:
import sys
!{sys.executable} -m pip install pafy
!{sys.executable} -m pip install youtube-dl
!{sys.executable} -m pip install python-dotenv
!{sys.executable} -m pip install facenet-pytorch

import zmq
import pafy
import numpy as np
import time
from os import getenv
import os
from dotenv import load_dotenv
import torch
import torchvision
import PIL.Image
import PIL.ImageDraw
import torchvision.transforms as transforms
import torch.nn.functional as F
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
import IPython
from jetcam.usb_camera import USBCamera
from facenet_pytorch import MTCNN

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()

camera = USBCamera(width=224, height=224, capture_device=0) # confirm the capture_device number
camera.running = True
print("camera created")

# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

# create image preview
camera_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 2)
model = model.to(device)
model.load_state_dict(torch.load('./model/my_model(1).pth'))
model.eval()

mtcnn = MTCNN(keep_all=True, device=device)

load_dotenv()
outframe = None
url = "https://youtu.be/7PYzSXHd6U4"

#defaults
active = "Live"
default_radius_size = getenv('DEFAULT_RADIUS')
default_radius_uom = getenv('DEFAULT_RADIUS_UOM')
default_duration = getenv('DEFAULT_DURATION')
default_duration_uom = getenv('DEFAULT_DURATION_UOM')
message = None

#working values
radius_size = getenv('DEFAULT_RADIUS')
radius_uom = getenv('DEFAULT_RADIUS_UOM')
duration = getenv('DEFAULT_DURATION')
duration_uom = getenv('DEFAULT_DURATION_UOM')

def preprocess(image):
    device = torch.device('cuda')
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

def change_source(change):
    global active
    active = change['new']
    
data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget])
])

state_widget = ipywidgets.ToggleButtons(options=['Live', 'Playback'], value='Live')
state_widget.observe(change_source, names='value')

all_widget = ipywidgets.VBox([
    state_widget,
    ipywidgets.VBox([data_collection_widget])
])
d = display(all_widget)
    
while True:
    frame = camera.value
    boxes, probs, landmarks = mtcnn.detect(frame, landmarks=True)
    for box, prob, ld in zip(boxes, probs, landmarks):
        draw = ImageDraw.Draw(frame)
        preprocessed = preprocess(image)
        output = model(preprocessed)
        output = F.softmax(output, dim=1).detach().cpu().numpy().flatten()
        category_index = output.argmax()
        draw.rectangle(box.tolist(), outline=(255,0,0), width=6)
        d.update(frame)
        print(output)

Microsoft Visual C++ Redistributable is not installed, this may lead to the DLL load failure.
                 It can be downloaded at https://aka.ms/vs/16/release/vc_redist.x64.exe


OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\bjarreau\Anaconda3\lib\site-packages\torch\lib\asmjit.dll" or one of its dependencies.